In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import tensorflow as tf
import time
import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from thesis.create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
)
from thesis.helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstraping_partitions_full_int_q,
    find_decision_threshold
    )

In [22]:
sample_rate = 16000

In [23]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=sample_rate, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.
Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [24]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [25]:
models_thresholds_and_metrics = {"SqueezeNet-Mel":{}}

In [26]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras"
baseline_model = keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "KB")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 184, 80, 1)]         0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 92, 40, 96)           4800      ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling2D)     (None, 45, 19, 96)           0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv2D)      (None, 45, 19, 16)           1552      ['maxpool1[0][0]']            
                                                                                              

In [30]:
print("\nTest dataset:")
(
    threshold,
    y_pred_val,
    _, 
    # non_overlap_patritions_f1_scores_test, 
    # bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_val_np, y_val_np, model_format="keras")

(
    _,
    y_pred_test,
    metrics_dict, 
    # non_overlap_patritions_f1_scores_test, 
    # bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras", threshold=threshold)


Test dataset:
44/44 [==============================] - 3s 69ms/step
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7348694
Accuracy: 99.20%
Recall: 99.36%
Precision: 98.32%
F1-score: 98.84%
F2-score: 99.19%
44/44 [==============================] - 3s 64ms/step
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7348694
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.61%


In [31]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["baseline"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

### Convert to TF Lite format

In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3qw_c4q3/assets


INFO:tensorflow:Assets written to: /tmp/tmp3qw_c4q3/assets
2024-08-27 20:48:22.434636: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-27 20:48:22.434692: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-27 20:48:22.434861: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp3qw_c4q3
2024-08-27 20:48:22.439684: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-27 20:48:22.439728: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp3qw_c4q3
2024-08-27 20:48:22.456246: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-27 20:48:22.657192: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp3qw_c4q3
2024-08-27 20:48:22.751969: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [38]:
print("Validation dataset:")
(
    threshold,
    y_pred_val,
    _, 
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    _,
    y_pred_test,
    metrics_dict, 
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite", threshold=threshold)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7348694
Accuracy: 99.20%
Recall: 99.36%
Precision: 98.32%
F1-score: 98.84%
F2-score: 99.19%

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7348694
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.61%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.007 seconds
Max: 0.013 seconds
Min: 0.006 seconds


Model file name:  spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz.tflite
File size: 2.789 Megabytes


In [39]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["tflite"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'tflite': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

In [40]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 2856.07 Kilobytes


### Dynamic range quantization

In [41]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpfk1w_euq/assets


INFO:tensorflow:Assets written to: /tmp/tmpfk1w_euq/assets
2024-08-27 20:51:56.250149: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-27 20:51:56.250198: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-27 20:51:56.250359: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfk1w_euq
2024-08-27 20:51:56.254625: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-27 20:51:56.254639: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpfk1w_euq
2024-08-27 20:51:56.267644: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-27 20:51:56.451109: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpfk1w_euq
2024-08-27 20:51:56.502602: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [42]:
print("Validation dataset:")
(
    threshold,
    y_pred_val,
    _, 
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    _,
    y_pred_test,
    metrics_dict, 
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite", threshold=threshold)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7340945
Accuracy: 99.20%
Recall: 99.36%
Precision: 98.32%
F1-score: 98.84%
F2-score: 99.19%

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.7340945
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.61%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.01 seconds
Max: 0.015 seconds
Min: 0.009 seconds


Model file name:  spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_drq.tflite
File size: 770.258 Kilobytes


In [43]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["DRQ"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'tflite': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'DRQ': {'threshold': '0.7340945',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

### Float16 quantization

In [44]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp66yfciu8/assets


INFO:tensorflow:Assets written to: /tmp/tmp66yfciu8/assets
2024-08-27 20:53:00.733087: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-27 20:53:00.733141: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-27 20:53:00.733306: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp66yfciu8
2024-08-27 20:53:00.739610: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-27 20:53:00.739718: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp66yfciu8
2024-08-27 20:53:00.760945: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-27 20:53:01.032512: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp66yfciu8
2024-08-27 20:53:01.162065: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [45]:
print("Validation dataset:")
(
    threshold,
    y_pred_val,
    _, 
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    _,
    y_pred_test,
    metrics_dict, 
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite", threshold=threshold)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.73482734
Accuracy: 99.20%
Recall: 99.36%
Precision: 98.32%
F1-score: 98.84%
F2-score: 99.19%

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.73482734
Accuracy: 98.56%
Recall: 99.34%
Precision: 96.40%
F1-score: 97.85%
F2-score: 98.61%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.007 seconds
Max: 0.027 seconds
Min: 0.007 seconds


Model file name:  spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float16q.tflite
File size: 1.409 Megabytes


In [46]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["float16"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'tflite': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'DRQ': {'threshold': '0.7340945',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'float16': {'threshold': '0.73482734',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

In [47]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 1442.977 Kilobytes


### Float fallback quantization

In [48]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]
    
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpn4q11pic/assets


INFO:tensorflow:Assets written to: /tmp/tmpn4q11pic/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-08-27 20:55:16.181441: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-27 20:55:16.181508: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-27 20:55:16.181769: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpn4q11pic
2024-08-27 20:55:16.187871: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-27 20:55:16.187917: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpn4q11pic
2024-08-27 20:55:16.211602: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-27 20:55

In [49]:
print("Validation dataset:")
(
    threshold,
    y_pred_val,
    _, 
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    _,
    y_pred_test,
    metrics_dict, 
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite", threshold=threshold)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.45703125
Accuracy: 99.06%
Recall: 99.58%
Precision: 97.71%
F1-score: 98.64%
F2-score: 99.09%

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Optimal threshold:  0.45703125
Accuracy: 98.49%
Recall: 99.56%
Precision: 96.00%
F1-score: 97.75%
F2-score: 98.58%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.01 seconds
Max: 0.027 seconds
Min: 0.008 seconds


Model file name:  spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float_fallback_q.tflite
File size: 0.789 Megabytes


In [ ]:
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "KB")

File size: 807.516 Kilobytes


In [50]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["float_fallback"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'tflite': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'DRQ': {'threshold': '0.7340945',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'float16': {'threshold': '0.73482734',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'float_fallback': {'threshold': '0.45703125',
   'accuracy': 0.98,
   'recall': 1.0,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

Full int

In [51]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_quant = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4tu37so5/assets


INFO:tensorflow:Assets written to: /tmp/tmp4tu37so5/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-08-27 20:56:30.754662: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-27 20:56:30.754711: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-27 20:56:30.754884: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4tu37so5
2024-08-27 20:56:30.761300: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-27 20:56:30.761329: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4tu37so5
2024-08-27 20:56:30.781151: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-27 20:56

In [52]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [53]:
import pathlib

tflite_models_dir = pathlib.Path("spectrogram_models_from_notebooks/squeezenet")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"squeezenet_spec_16kHz_full_int_q.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

826920

In [55]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_val_np)
threshold = find_decision_threshold(y_val_np, predictions, model_format="tflite")
print(threshold)
_ = evaluate_prediction(y_val_np, predictions, model_format="tflite", optimal_threshold=threshold)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_test_np)
metrics_dict = evaluate_prediction(y_test_np, predictions, model_format="tflite", optimal_threshold=threshold)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_quant_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_quant_file), "KB")

Validation dataset:
117.0
Optimal threshold:  117.0
Accuracy: 98.91%
Recall: 99.58%
Precision: 97.31%
F1-score: 98.43%
F2-score: 98.96%

Test dataset:
Optimal threshold:  117.0
Accuracy: 98.64%
Recall: 99.78%
Precision: 96.21%
F1-score: 97.96%
F2-score: 98.74%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.013 seconds
Max: 0.026 seconds
Min: 0.009 seconds
File size: 807.539 Kilobytes


In [56]:
models_thresholds_and_metrics["SqueezeNet-Mel"]["full int"] = {"threshold": str(threshold), **metrics_dict}
models_thresholds_and_metrics

{'SqueezeNet-Mel': {'baseline': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'tflite': {'threshold': '0.7348694',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'DRQ': {'threshold': '0.7340945',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'float16': {'threshold': '0.73482734',
   'accuracy': 0.99,
   'recall': 0.99,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'float_fallback': {'threshold': '0.45703125',
   'accuracy': 0.98,
   'recall': 1.0,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99},
  'full int': {'threshold': '117.0',
   'accuracy': 0.99,
   'recall': 1.0,
   'precision': 0.96,
   'f1-score': 0.98,
   'f2-score': 0.99}}}

In [58]:
import pandas as pd
import json

In [61]:
pd.DataFrame(models_thresholds_and_metrics["SqueezeNet-Mel"]).T

,threshold,accuracy,recall,precision,f1-score,f2-score
baseline,0.7348694,0.99,0.99,0.96,0.98,0.99
tflite,0.7348694,0.99,0.99,0.96,0.98,0.99
DRQ,0.7340945,0.99,0.99,0.96,0.98,0.99
float16,0.73482734,0.99,0.99,0.96,0.98,0.99
float_fallback,0.45703125,0.98,1.0,0.96,0.98,0.99
full int,117.0,0.99,1.0,0.96,0.98,0.99


In [65]:
with open('thesis/thresholds_and_metrics_json/squeezenet_mel.json', 'w') as f:
    json.dump(models_thresholds_and_metrics, f)